In [1]:
# Importar librerias
import pandas as pd
from collections import Counter
import matplotlib.pyplot as plt
from pandas.core.arrays import categorical # agregar class pandas.Categorical(values, categories=None, ordered=None, dtype=None, fastpath=False, copy=True)
from sklearn.pipeline import Pipeline #agregar para ejercicio 3a
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, StandardScaler
import re

In [2]:
# Importar dataframe 1 "Events downtime"
downtime = pd.read_csv("/content/Ensamble 100D production history DB1 22-23-24.csv")

In [3]:
# Visualizar df
downtime

,EventID,WorkGroup,AssetName,DownStart,DownEnd,Total Duration,Shift Scheduled Duration,Total Scheduled Duration,Reason,Code,ShiftNumber,ScheduleID,OEE
0,1052248,Ensamble 100D,Bench 100D,1/6/2022,1/6/2022,0:00:12,0:00:12,0:00:12,Falla 49: Lifter MarkerPosn. Superior,NaN,1,25579,1.00
1,1052250,Ensamble 100D,Bench 100D,1/6/2022,1/6/2022,0:00:02,0:00:02,0:00:02,Falla 58: C-Clip Posn. Extendida,NaN,1,25579,1.00
2,1052253,Ensamble 100D,Bench 100D,1/6/2022,1/6/2022,0:00:01,0:00:01,0:00:01,Falla189. Paremtro Close lenght Tolerancia Act...,NaN,1,25579,1.00
3,1052265,Ensamble 100D,Bench 100D,1/6/2022,1/6/2022,0:00:01,0:00:01,0:00:01,Falla 49: Lifter MarkerPosn. Superior,NaN,1,25579,1.00
4,1052274,Ensamble 100D,Bench 100D,1/6/2022,1/6/2022,0:00:08,0:00:08,0:00:08,Falla 102: Disponible,NaN,1,25579,1.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
52347,1717122,Ensamble 100D,Bench 100D,5/7/2024,5/7/2024,0:00:01,0:00:01,0:00:01,Falla35. Servo Pin Izquierdo Longitud Fallo,NaN,2,59167,1.00
52348,1717133,Ensamble 100D,Bench 100D,5/7/2024,5/7/2024,0:00:56,0:00:56,0:00:56,Ing - ajuste,NaN,2,59167,1.00
52349,1717135,Ensamble 100D,Bench 100D,5/7/2024,5/7/2024,0:23:29,0:23:29,0:23:29,Ing - En Proceso: ajuste,NaN,2,59167,0.96
52350,1717226,Ensamble 100D,Bench 100D,5/7/2024,5/7/2024,0:00:01,0:00:01,0:00:01,Falla 12. Scanner de Seguridad Interrumpido Du...,NaN,2,59167,1.00


In [4]:
# contar  valores nulos de dataframe "Events downtime"
downtime.isnull().sum().sort_values(ascending=True)

EventID                         0
WorkGroup                       0
AssetName                       0
DownStart                       0
DownEnd                         0
Total Duration                  0
Shift Scheduled Duration        0
Total Scheduled Duration        0
ShiftNumber                     0
ScheduleID                      0
OEE                             0
Reason                        332
Code                        52352
dtype: int64

In [5]:
# limpiar columnas que no se van a utilizar de dataframe "Events downtime"
downtime.drop(["WorkGroup","Code", "Shift Scheduled Duration", "Total Scheduled Duration"], axis = 1, inplace = True)


In [6]:
downtime

,EventID,AssetName,DownStart,DownEnd,Total Duration,Reason,ShiftNumber,ScheduleID,OEE
0,1052248,Bench 100D,1/6/2022,1/6/2022,0:00:12,Falla 49: Lifter MarkerPosn. Superior,1,25579,1.00
1,1052250,Bench 100D,1/6/2022,1/6/2022,0:00:02,Falla 58: C-Clip Posn. Extendida,1,25579,1.00
2,1052253,Bench 100D,1/6/2022,1/6/2022,0:00:01,Falla189. Paremtro Close lenght Tolerancia Act...,1,25579,1.00
3,1052265,Bench 100D,1/6/2022,1/6/2022,0:00:01,Falla 49: Lifter MarkerPosn. Superior,1,25579,1.00
4,1052274,Bench 100D,1/6/2022,1/6/2022,0:00:08,Falla 102: Disponible,1,25579,1.00
...,...,...,...,...,...,...,...,...,...
52347,1717122,Bench 100D,5/7/2024,5/7/2024,0:00:01,Falla35. Servo Pin Izquierdo Longitud Fallo,2,59167,1.00
52348,1717133,Bench 100D,5/7/2024,5/7/2024,0:00:56,Ing - ajuste,2,59167,1.00
52349,1717135,Bench 100D,5/7/2024,5/7/2024,0:23:29,Ing - En Proceso: ajuste,2,59167,0.96
52350,1717226,Bench 100D,5/7/2024,5/7/2024,0:00:01,Falla 12. Scanner de Seguridad Interrumpido Du...,2,59167,1.00


In [7]:
# eliminar valores nulos o NAN de tabla downtime

downtime.dropna(inplace=True)
downtime


,EventID,AssetName,DownStart,DownEnd,Total Duration,Reason,ShiftNumber,ScheduleID,OEE
0,1052248,Bench 100D,1/6/2022,1/6/2022,0:00:12,Falla 49: Lifter MarkerPosn. Superior,1,25579,1.00
1,1052250,Bench 100D,1/6/2022,1/6/2022,0:00:02,Falla 58: C-Clip Posn. Extendida,1,25579,1.00
2,1052253,Bench 100D,1/6/2022,1/6/2022,0:00:01,Falla189. Paremtro Close lenght Tolerancia Act...,1,25579,1.00
3,1052265,Bench 100D,1/6/2022,1/6/2022,0:00:01,Falla 49: Lifter MarkerPosn. Superior,1,25579,1.00
4,1052274,Bench 100D,1/6/2022,1/6/2022,0:00:08,Falla 102: Disponible,1,25579,1.00
...,...,...,...,...,...,...,...,...,...
52347,1717122,Bench 100D,5/7/2024,5/7/2024,0:00:01,Falla35. Servo Pin Izquierdo Longitud Fallo,2,59167,1.00
52348,1717133,Bench 100D,5/7/2024,5/7/2024,0:00:56,Ing - ajuste,2,59167,1.00
52349,1717135,Bench 100D,5/7/2024,5/7/2024,0:23:29,Ing - En Proceso: ajuste,2,59167,0.96
52350,1717226,Bench 100D,5/7/2024,5/7/2024,0:00:01,Falla 12. Scanner de Seguridad Interrumpido Du...,2,59167,1.00


In [8]:
# Renombra la columna Total Duration a Total_Duration

downtime.rename(columns={'Total Duration': 'Total_Duration'}, inplace=True)
downtime


,EventID,AssetName,DownStart,DownEnd,Total_Duration,Reason,ShiftNumber,ScheduleID,OEE
0,1052248,Bench 100D,1/6/2022,1/6/2022,0:00:12,Falla 49: Lifter MarkerPosn. Superior,1,25579,1.00
1,1052250,Bench 100D,1/6/2022,1/6/2022,0:00:02,Falla 58: C-Clip Posn. Extendida,1,25579,1.00
2,1052253,Bench 100D,1/6/2022,1/6/2022,0:00:01,Falla189. Paremtro Close lenght Tolerancia Act...,1,25579,1.00
3,1052265,Bench 100D,1/6/2022,1/6/2022,0:00:01,Falla 49: Lifter MarkerPosn. Superior,1,25579,1.00
4,1052274,Bench 100D,1/6/2022,1/6/2022,0:00:08,Falla 102: Disponible,1,25579,1.00
...,...,...,...,...,...,...,...,...,...
52347,1717122,Bench 100D,5/7/2024,5/7/2024,0:00:01,Falla35. Servo Pin Izquierdo Longitud Fallo,2,59167,1.00
52348,1717133,Bench 100D,5/7/2024,5/7/2024,0:00:56,Ing - ajuste,2,59167,1.00
52349,1717135,Bench 100D,5/7/2024,5/7/2024,0:23:29,Ing - En Proceso: ajuste,2,59167,0.96
52350,1717226,Bench 100D,5/7/2024,5/7/2024,0:00:01,Falla 12. Scanner de Seguridad Interrumpido Du...,2,59167,1.00


In [9]:
# convertir columna Total_Duration a segundos, el formato actual es HH:MM:ss, las horas no son horas dia

def convert_to_seconds(time_str):
  hours, minutes, seconds = map(int, time_str.split(':'))
  return hours * 3600 + minutes * 60 + seconds

downtime['Total_Duration'] = downtime['Total_Duration'].apply(convert_to_seconds)


In [10]:
downtime

,EventID,AssetName,DownStart,DownEnd,Total_Duration,Reason,ShiftNumber,ScheduleID,OEE
0,1052248,Bench 100D,1/6/2022,1/6/2022,12,Falla 49: Lifter MarkerPosn. Superior,1,25579,1.00
1,1052250,Bench 100D,1/6/2022,1/6/2022,2,Falla 58: C-Clip Posn. Extendida,1,25579,1.00
2,1052253,Bench 100D,1/6/2022,1/6/2022,1,Falla189. Paremtro Close lenght Tolerancia Act...,1,25579,1.00
3,1052265,Bench 100D,1/6/2022,1/6/2022,1,Falla 49: Lifter MarkerPosn. Superior,1,25579,1.00
4,1052274,Bench 100D,1/6/2022,1/6/2022,8,Falla 102: Disponible,1,25579,1.00
...,...,...,...,...,...,...,...,...,...
52347,1717122,Bench 100D,5/7/2024,5/7/2024,1,Falla35. Servo Pin Izquierdo Longitud Fallo,2,59167,1.00
52348,1717133,Bench 100D,5/7/2024,5/7/2024,56,Ing - ajuste,2,59167,1.00
52349,1717135,Bench 100D,5/7/2024,5/7/2024,1409,Ing - En Proceso: ajuste,2,59167,0.96
52350,1717226,Bench 100D,5/7/2024,5/7/2024,1,Falla 12. Scanner de Seguridad Interrumpido Du...,2,59167,1.00


In [11]:
# Separar columna DownStart de dia mes y año
# Separar columna DownStart en  tres columnas mas  una que represente el dia, otra el mes y otra el año

downtime['DownStart_dia'] = pd.to_datetime(downtime['DownStart']).dt.day
downtime['DownStart_mes'] = pd.to_datetime(downtime['DownStart']).dt.month
downtime['DownStart_año'] = pd.to_datetime(downtime['DownStart']).dt.year
downtime


,EventID,AssetName,DownStart,DownEnd,Total_Duration,Reason,ShiftNumber,ScheduleID,OEE,DownStart_dia,DownStart_mes,DownStart_año
0,1052248,Bench 100D,1/6/2022,1/6/2022,12,Falla 49: Lifter MarkerPosn. Superior,1,25579,1.00,6,1,2022
1,1052250,Bench 100D,1/6/2022,1/6/2022,2,Falla 58: C-Clip Posn. Extendida,1,25579,1.00,6,1,2022
2,1052253,Bench 100D,1/6/2022,1/6/2022,1,Falla189. Paremtro Close lenght Tolerancia Act...,1,25579,1.00,6,1,2022
3,1052265,Bench 100D,1/6/2022,1/6/2022,1,Falla 49: Lifter MarkerPosn. Superior,1,25579,1.00,6,1,2022
4,1052274,Bench 100D,1/6/2022,1/6/2022,8,Falla 102: Disponible,1,25579,1.00,6,1,2022
...,...,...,...,...,...,...,...,...,...,...,...,...
52347,1717122,Bench 100D,5/7/2024,5/7/2024,1,Falla35. Servo Pin Izquierdo Longitud Fallo,2,59167,1.00,7,5,2024
52348,1717133,Bench 100D,5/7/2024,5/7/2024,56,Ing - ajuste,2,59167,1.00,7,5,2024
52349,1717135,Bench 100D,5/7/2024,5/7/2024,1409,Ing - En Proceso: ajuste,2,59167,0.96,7,5,2024
52350,1717226,Bench 100D,5/7/2024,5/7/2024,1,Falla 12. Scanner de Seguridad Interrumpido Du...,2,59167,1.00,7,5,2024


In [12]:
# Filtrar las filas que contienen 'Falla' o 'Fault' o 'Advertencia' al inicio en la columna 'Reason'
filtered_downtime = downtime[downtime['Reason'].str.contains('^(Falla|Fault|Advertencia)')]

# Reasignar el DataFrame original con el filtrado
downtime = filtered_downtime

# Si deseas resetear los índices del DataFrame
# downtime.reset_index(drop=True, inplace=True)

# Imprimir el DataFrame resultante
print(downtime)


       EventID   AssetName DownStart   DownEnd  Total_Duration  \
0      1052248  Bench 100D  1/6/2022  1/6/2022              12   
1      1052250  Bench 100D  1/6/2022  1/6/2022               2   
2      1052253  Bench 100D  1/6/2022  1/6/2022               1   
3      1052265  Bench 100D  1/6/2022  1/6/2022               1   
4      1052274  Bench 100D  1/6/2022  1/6/2022               8   
...        ...         ...       ...       ...             ...   
52344  1717115  Bench 100D  5/7/2024  5/7/2024               1   
52345  1717120  Bench 100D  5/7/2024  5/7/2024               1   
52346  1717121  Bench 100D  5/7/2024  5/7/2024               2   
52347  1717122  Bench 100D  5/7/2024  5/7/2024               1   
52350  1717226  Bench 100D  5/7/2024  5/7/2024               1   

                                                  Reason  ShiftNumber  \
0                  Falla 49: Lifter MarkerPosn. Superior            1   
1                       Falla 58: C-Clip Posn. Extendida     

<ipython-input-12-e23c3cb4e99f>:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  filtered_downtime = downtime[downtime['Reason'].str.contains('^(Falla|Fault|Advertencia)')]


In [13]:
# Definir una función para extraer el número de falla
def extract_fault_number(reason):
    # Usar expresión regular para buscar el número de falla
    match = re.search(r'(Falla|Fault|Advertencia)[:.]*\s*(\d+)', reason)
    if match:
        return match.group(2)  # Devolver el segundo grupo que contiene el número de falla
    else:
        return None

# Aplicar la función a la columna "Reason" para extraer el número de falla
downtime["Fault_Number"] = downtime["Reason"].apply(extract_fault_number)

<ipython-input-13-b1d297b5528a>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downtime["Fault_Number"] = downtime["Reason"].apply(extract_fault_number)


In [14]:
downtime

,EventID,AssetName,DownStart,DownEnd,Total_Duration,Reason,ShiftNumber,ScheduleID,OEE,DownStart_dia,DownStart_mes,DownStart_año,Fault_Number
0,1052248,Bench 100D,1/6/2022,1/6/2022,12,Falla 49: Lifter MarkerPosn. Superior,1,25579,1.0,6,1,2022,49
1,1052250,Bench 100D,1/6/2022,1/6/2022,2,Falla 58: C-Clip Posn. Extendida,1,25579,1.0,6,1,2022,58
2,1052253,Bench 100D,1/6/2022,1/6/2022,1,Falla189. Paremtro Close lenght Tolerancia Act...,1,25579,1.0,6,1,2022,189
3,1052265,Bench 100D,1/6/2022,1/6/2022,1,Falla 49: Lifter MarkerPosn. Superior,1,25579,1.0,6,1,2022,49
4,1052274,Bench 100D,1/6/2022,1/6/2022,8,Falla 102: Disponible,1,25579,1.0,6,1,2022,102
...,...,...,...,...,...,...,...,...,...,...,...,...,...
52344,1717115,Bench 100D,5/7/2024,5/7/2024,1,Falla35. Servo Pin Izquierdo Longitud Fallo,2,59167,1.0,7,5,2024,35
52345,1717120,Bench 100D,5/7/2024,5/7/2024,1,Falla35. Servo Pin Izquierdo Longitud Fallo,2,59167,1.0,7,5,2024,35
52346,1717121,Bench 100D,5/7/2024,5/7/2024,2,Falla 12. Scanner de Seguridad Interrumpido Du...,2,59167,1.0,7,5,2024,12
52347,1717122,Bench 100D,5/7/2024,5/7/2024,1,Falla35. Servo Pin Izquierdo Longitud Fallo,2,59167,1.0,7,5,2024,35


In [15]:
# Definir el diccionario de categorías
fault_categories = {
  "Servo Support Faults": ["14", "17", "19", "23", "26", "28", "29", "32", "33", "34", "35", "151"],
  "Servo Pin Faults": ["20"],
  "Pin Position Faults": ["61", "63", "64", "65", "66", "67", "68", "69", "70", "72", "74", "75", "77", "78", "79", "80", "85", "86", "88"],
  "Position Sensor Faults": ["44", "45", "46", "47", "49", "50", "51", "52", "53", "54", "55", "58", "59","90","189"],
  "External Device Faults": ["56","183"],
  "Relay Faults": ["1", "207", "208"],
  "Scanner Faults": ["12", "96"],
  "Electrical Faults": ["2", "97"],
  "Other Faults": ["104","102"]
}

# Función para mapear los valores de la columna "Reason" a las categorías correspondientes
def categorize_reason(reason):
    for category, faults in fault_categories.items():
        if reason in faults:
            return category
    return "Other Faults"  # Si el motivo no coincide con ninguna categoría de otros

# Aplicar la función de categorización a la columna "Reason"
downtime['Category'] = downtime['Fault_Number'].apply(categorize_reason)

# Imprimir el DataFrame con las categorías agregadas
downtime


<ipython-input-15-92aab13925a9>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downtime['Category'] = downtime['Fault_Number'].apply(categorize_reason)


,EventID,AssetName,DownStart,DownEnd,Total_Duration,Reason,ShiftNumber,ScheduleID,OEE,DownStart_dia,DownStart_mes,DownStart_año,Fault_Number,Category
0,1052248,Bench 100D,1/6/2022,1/6/2022,12,Falla 49: Lifter MarkerPosn. Superior,1,25579,1.0,6,1,2022,49,Position Sensor Faults
1,1052250,Bench 100D,1/6/2022,1/6/2022,2,Falla 58: C-Clip Posn. Extendida,1,25579,1.0,6,1,2022,58,Position Sensor Faults
2,1052253,Bench 100D,1/6/2022,1/6/2022,1,Falla189. Paremtro Close lenght Tolerancia Act...,1,25579,1.0,6,1,2022,189,Position Sensor Faults
3,1052265,Bench 100D,1/6/2022,1/6/2022,1,Falla 49: Lifter MarkerPosn. Superior,1,25579,1.0,6,1,2022,49,Position Sensor Faults
4,1052274,Bench 100D,1/6/2022,1/6/2022,8,Falla 102: Disponible,1,25579,1.0,6,1,2022,102,Other Faults
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52344,1717115,Bench 100D,5/7/2024,5/7/2024,1,Falla35. Servo Pin Izquierdo Longitud Fallo,2,59167,1.0,7,5,2024,35,Servo Support Faults
52345,1717120,Bench 100D,5/7/2024,5/7/2024,1,Falla35. Servo Pin Izquierdo Longitud Fallo,2,59167,1.0,7,5,2024,35,Servo Support Faults
52346,1717121,Bench 100D,5/7/2024,5/7/2024,2,Falla 12. Scanner de Seguridad Interrumpido Du...,2,59167,1.0,7,5,2024,12,Scanner Faults
52347,1717122,Bench 100D,5/7/2024,5/7/2024,1,Falla35. Servo Pin Izquierdo Longitud Fallo,2,59167,1.0,7,5,2024,35,Servo Support Faults


In [16]:
# Definir el diccionario de categorías
fault_categories = {
  "Servo Support Faults": ["14", "17", "19", "23", "26", "28", "29", "32", "33", "34", "35", "151","146","93","92","91","157"],
  "Servo Pin Faults": ["20","148","197"],
  "Pin Position Faults": ["61", "63", "64", "65", "66", "67", "68", "69", "70", "72", "74", "75", "77", "78", "79", "80", "85", "86", "88","153"],
  "Position Sensor Faults": ["43","44", "45", "46", "47", "49", "50", "51", "52", "53", "54", "55", "58", "59","90","189"],
  "External Device Faults": ["56","183","109","161"],
  "Relay Faults": ["1", "207", "208", "209"],
  "Scanner Faults": ["12", "96"],
  "Electrical Faults": ["2", "97"],
  "Invalid Parameters": ["193","180","178","165"],
  "Other Faults": ["104","102", "193", "98", "93", "91", "125", "101","13"]

}

# Función para mapear los valores de la columna "Reason" a las categorías correspondientes
def categorize_reason(reason):
    for category, faults in fault_categories.items():
        if reason in faults:
            return category
    return "Other Faults"  # Si el motivo no coincide con ninguna categoría de otros

# Aplicar la función de categorización a la columna "Reason"
downtime['Category'] = downtime['Fault_Number'].apply(categorize_reason)

# Imprimir el DataFrame con las categorías agregadas
downtime


<ipython-input-16-67e30f9ee899>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downtime['Category'] = downtime['Fault_Number'].apply(categorize_reason)


,EventID,AssetName,DownStart,DownEnd,Total_Duration,Reason,ShiftNumber,ScheduleID,OEE,DownStart_dia,DownStart_mes,DownStart_año,Fault_Number,Category
0,1052248,Bench 100D,1/6/2022,1/6/2022,12,Falla 49: Lifter MarkerPosn. Superior,1,25579,1.0,6,1,2022,49,Position Sensor Faults
1,1052250,Bench 100D,1/6/2022,1/6/2022,2,Falla 58: C-Clip Posn. Extendida,1,25579,1.0,6,1,2022,58,Position Sensor Faults
2,1052253,Bench 100D,1/6/2022,1/6/2022,1,Falla189. Paremtro Close lenght Tolerancia Act...,1,25579,1.0,6,1,2022,189,Position Sensor Faults
3,1052265,Bench 100D,1/6/2022,1/6/2022,1,Falla 49: Lifter MarkerPosn. Superior,1,25579,1.0,6,1,2022,49,Position Sensor Faults
4,1052274,Bench 100D,1/6/2022,1/6/2022,8,Falla 102: Disponible,1,25579,1.0,6,1,2022,102,Other Faults
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52344,1717115,Bench 100D,5/7/2024,5/7/2024,1,Falla35. Servo Pin Izquierdo Longitud Fallo,2,59167,1.0,7,5,2024,35,Servo Support Faults
52345,1717120,Bench 100D,5/7/2024,5/7/2024,1,Falla35. Servo Pin Izquierdo Longitud Fallo,2,59167,1.0,7,5,2024,35,Servo Support Faults
52346,1717121,Bench 100D,5/7/2024,5/7/2024,2,Falla 12. Scanner de Seguridad Interrumpido Du...,2,59167,1.0,7,5,2024,12,Scanner Faults
52347,1717122,Bench 100D,5/7/2024,5/7/2024,1,Falla35. Servo Pin Izquierdo Longitud Fallo,2,59167,1.0,7,5,2024,35,Servo Support Faults


In [17]:
# Contar la frecuencia de cada valor en la columna 'DayOfWeek'
frequency_counts = downtime['Category'].value_counts()

# Mostrar la frecuencia de cada valor
print("Frecuencia de cada valor en la columna 'Category':")
print(frequency_counts)

Frecuencia de cada valor en la columna 'Category':
Category
Servo Support Faults      18871
Scanner Faults            13894
Position Sensor Faults     8114
Pin Position Faults        5912
Relay Faults               1660
Other Faults                977
Invalid Parameters          917
External Device Faults      467
Electrical Faults           108
Servo Pin Faults             35
Name: count, dtype: int64


In [18]:
# Agrega una columna en el DF de downtime donde evalue la columna Category y le asigne un numero para cada categoria, esta nueva columna se llamara "Category_Number" y sera del tipo int y comenzara a partir del 0

# Crear un diccionario que mapee las categorías a números
category_to_number = {
    "Servo Support Faults": 0,
    "Servo Pin Faults": 1,
    "Pin Position Faults": 2,
    "Position Sensor Faults": 3,
    "External Device Faults": 4,
    "Relay Faults": 5,
    "Scanner Faults": 6,
    "Electrical Faults": 7,
    "Invalid Parameters": 8,
    "Other Faults": 9
}

# Agregar una nueva columna "Category_Number" al DataFrame
downtime["Category_Number"] = downtime["Category"].map(category_to_number).astype(int)

# Imprimir el DataFrame con la nueva columna
print(downtime)


       EventID   AssetName DownStart   DownEnd  Total_Duration  \
0      1052248  Bench 100D  1/6/2022  1/6/2022              12   
1      1052250  Bench 100D  1/6/2022  1/6/2022               2   
2      1052253  Bench 100D  1/6/2022  1/6/2022               1   
3      1052265  Bench 100D  1/6/2022  1/6/2022               1   
4      1052274  Bench 100D  1/6/2022  1/6/2022               8   
...        ...         ...       ...       ...             ...   
52344  1717115  Bench 100D  5/7/2024  5/7/2024               1   
52345  1717120  Bench 100D  5/7/2024  5/7/2024               1   
52346  1717121  Bench 100D  5/7/2024  5/7/2024               2   
52347  1717122  Bench 100D  5/7/2024  5/7/2024               1   
52350  1717226  Bench 100D  5/7/2024  5/7/2024               1   

                                                  Reason  ShiftNumber  \
0                  Falla 49: Lifter MarkerPosn. Superior            1   
1                       Falla 58: C-Clip Posn. Extendida     

<ipython-input-18-e45b3244ebc1>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downtime["Category_Number"] = downtime["Category"].map(category_to_number).astype(int)


In [19]:
#imprime el DF con columna nueva
downtime

,EventID,AssetName,DownStart,DownEnd,Total_Duration,Reason,ShiftNumber,ScheduleID,OEE,DownStart_dia,DownStart_mes,DownStart_año,Fault_Number,Category,Category_Number
0,1052248,Bench 100D,1/6/2022,1/6/2022,12,Falla 49: Lifter MarkerPosn. Superior,1,25579,1.0,6,1,2022,49,Position Sensor Faults,3
1,1052250,Bench 100D,1/6/2022,1/6/2022,2,Falla 58: C-Clip Posn. Extendida,1,25579,1.0,6,1,2022,58,Position Sensor Faults,3
2,1052253,Bench 100D,1/6/2022,1/6/2022,1,Falla189. Paremtro Close lenght Tolerancia Act...,1,25579,1.0,6,1,2022,189,Position Sensor Faults,3
3,1052265,Bench 100D,1/6/2022,1/6/2022,1,Falla 49: Lifter MarkerPosn. Superior,1,25579,1.0,6,1,2022,49,Position Sensor Faults,3
4,1052274,Bench 100D,1/6/2022,1/6/2022,8,Falla 102: Disponible,1,25579,1.0,6,1,2022,102,Other Faults,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52344,1717115,Bench 100D,5/7/2024,5/7/2024,1,Falla35. Servo Pin Izquierdo Longitud Fallo,2,59167,1.0,7,5,2024,35,Servo Support Faults,0
52345,1717120,Bench 100D,5/7/2024,5/7/2024,1,Falla35. Servo Pin Izquierdo Longitud Fallo,2,59167,1.0,7,5,2024,35,Servo Support Faults,0
52346,1717121,Bench 100D,5/7/2024,5/7/2024,2,Falla 12. Scanner de Seguridad Interrumpido Du...,2,59167,1.0,7,5,2024,12,Scanner Faults,6
52347,1717122,Bench 100D,5/7/2024,5/7/2024,1,Falla35. Servo Pin Izquierdo Longitud Fallo,2,59167,1.0,7,5,2024,35,Servo Support Faults,0


In [20]:
# prompt: Imprime el tipo de dato para cada columna

for column in downtime.columns:
    print(f"{column}: {downtime[column].dtype}")


EventID: int64
AssetName: object
DownStart: object
DownEnd: object
Total_Duration: int64
Reason: object
ShiftNumber: int64
ScheduleID: int64
OEE: float64
DownStart_dia: int32
DownStart_mes: int32
DownStart_año: int32
Fault_Number: object
Category: object
Category_Number: int64


In [21]:
# Transformar las columnas DownStart y DownEnd a tipo Fechas, la columna Fault_Number a int, muestra el data frame downtime

import pandas as pd
# Transformar las columnas DownStart y DownEnd a tipo Fechas
downtime['DownStart'] = pd.to_datetime(downtime['DownStart'])
downtime['DownEnd'] = pd.to_datetime(downtime['DownEnd'])

# Transformar la columna Fault_Number a int
downtime['Fault_Number'] = downtime['Fault_Number'].astype(int)

# Mostrar el DataFrame downtime
print(downtime)


       EventID   AssetName  DownStart    DownEnd  Total_Duration  \
0      1052248  Bench 100D 2022-01-06 2022-01-06              12   
1      1052250  Bench 100D 2022-01-06 2022-01-06               2   
2      1052253  Bench 100D 2022-01-06 2022-01-06               1   
3      1052265  Bench 100D 2022-01-06 2022-01-06               1   
4      1052274  Bench 100D 2022-01-06 2022-01-06               8   
...        ...         ...        ...        ...             ...   
52344  1717115  Bench 100D 2024-05-07 2024-05-07               1   
52345  1717120  Bench 100D 2024-05-07 2024-05-07               1   
52346  1717121  Bench 100D 2024-05-07 2024-05-07               2   
52347  1717122  Bench 100D 2024-05-07 2024-05-07               1   
52350  1717226  Bench 100D 2024-05-07 2024-05-07               1   

                                                  Reason  ShiftNumber  \
0                  Falla 49: Lifter MarkerPosn. Superior            1   
1                       Falla 58: C-C

<ipython-input-21-1a17fc92f5f2>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downtime['DownStart'] = pd.to_datetime(downtime['DownStart'])
<ipython-input-21-1a17fc92f5f2>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downtime['DownEnd'] = pd.to_datetime(downtime['DownEnd'])
<ipython-input-21-1a17fc92f5f2>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://

In [22]:
# Verifica valores Nulos en las columnas y NAN

downtime.isnull().sum()


EventID            0
AssetName          0
DownStart          0
DownEnd            0
Total_Duration     0
Reason             0
ShiftNumber        0
ScheduleID         0
OEE                0
DownStart_dia      0
DownStart_mes      0
DownStart_año      0
Fault_Number       0
Category           0
Category_Number    0
dtype: int64

In [23]:
# Agrupa las columnas de la siguiente manera : EventID, ScheduleID, AssetName, DownStart, DownEnd, Reason, Category, DownStart_dia, DownStart_mes, DownStart_año, Total_Duration, Fault_Number, Category_Number, OEE

downtime = downtime.groupby(['EventID', 'ScheduleID', 'AssetName', 'DownStart', 'DownEnd', 'Reason', 'Category', 'DownStart_dia', 'DownStart_mes', 'DownStart_año', 'Total_Duration', 'Fault_Number', 'Category_Number','ShiftNumber']).agg({'OEE': 'mean'})

print(downtime)


                                                                                                                                                                                                                                 OEE
EventID ScheduleID AssetName  DownStart  DownEnd    Reason                                             Category               DownStart_dia DownStart_mes DownStart_año Total_Duration Fault_Number Category_Number ShiftNumber     
1052248 25579      Bench 100D 2022-01-06 2022-01-06 Falla 49: Lifter MarkerPosn. Superior              Position Sensor Faults 6             1             2022          12             49           3               1            1.0
1052250 25579      Bench 100D 2022-01-06 2022-01-06 Falla 58: C-Clip Posn. Extendida                   Position Sensor Faults 6             1             2022          2              58           3               1            1.0
1052253 25579      Bench 100D 2022-01-06 2022-01-06 Falla189. Paremtro Close lenght 

In [24]:
# Guarda el data set depurado
downtime.to_csv("/content/downtime.csv", index= True, header = True)
